In [1]:
import os
import redis
import hashlib
from metaflow import FlowSpec, step, S3
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle

# Ejemplo de predicción de modelo usando Redis

Este notebook muestra cómo obtener predicciones de un modelo que produce predicciones en lotes. Las predicciones se cargaron en Redis. 

En este notebook, recuperamos las predicciones del modelo almacenadas en Redis. Los datos deben ser leídos, convertidos a cadenas y luego hasheados. Con este enfoque, podemos verificar si los datos existen en Redis y recuperar la predicción correspondiente. En caso de que los datos no existan, asignamos un valor de cero. Esta estrategia simula cómo podría comportarse un servicio en producción ante casos no contemplados.

La ventaja de utilizar Redis en este contexto radica en su capacidad para almacenar datos de forma eficiente en memoria, lo que permite un acceso rápido a las predicciones previamente calculadas. Además, Redis ofrece funcionalidades de almacenamiento en caché y persistencia de datos, lo que garantiza la disponibilidad y la integridad de las predicciones incluso en entornos de producción de alta demanda.

In [2]:
# Conectamos al servidor redis (asegúrate de que el docker compose esté corriendo)
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Configuración de las credenciales de acceso a AWS S3 (minio)
os.environ['AWS_ACCESS_KEY_ID'] = "minio"
os.environ['AWS_SECRET_ACCESS_KEY'] = "minio123"
os.environ['AWS_ENDPOINT_URL_S3'] = "http://localhost:9000"

In [3]:
# Obtención de datos para prueba

df = pd.read_csv("./breast_cancer.csv", header=None)

# Sampleamos 100 valores al azar
df_temp = df.sample(50)

test_values = df_temp.values.tolist()

s3 = S3(s3root="s3://amqtp/")
scaler_obj = s3.get("scaler.pkl")
with open(scaler_obj.path, 'rb') as f:
    scaler = pickle.load(f)

# Aplicamos el scaler a los datos
scaled_values = scaler.transform(df_temp)

print(scaled_values)

# Conversión de valores a cadenas y hash
# Esto debería implementarse en el pipeline. Dado que los números de punto flotante pueden 
# presentar problemas debido a pequeñas variaciones, se podría considerar redondearlos.
string_values = [' '.join(map(str, sublist)) for sublist in scaled_values]
hashed_values = [hashlib.sha256(substring.encode()).hexdigest() for substring in string_values]

# Inicializamos un diccionario para almacenar las salidas del modelo
model_outputs = {}

# Obtenemos las predicciones almacenadas en Redis
for hash_key in hashed_values:
    model_outputs[hash_key] = r.hgetall(f"predictions:{hash_key}")

# Reemplazo de valores nulos con un valor predeterminado
# Esto es necesario porque, en caso de una nueva entrada que el modelo no haya visto durante 
# el procesamiento por lotes, necesitamos proporcionar una salida. Esta entrada podría ser
# encolada para su posterior procesamiento en lotes.
# model_output = [value if value is not None else "jjjj" for value in model_output]

[[-1.29416847 -0.55103206 -1.26206099 ... -0.51153826  0.15906632
   0.84091844]
 [ 1.81914568  2.41657614  1.97220321 ...  2.36910289  1.93445849
   2.31269552]
 [-1.04666651 -0.99296571 -1.02091333 ... -0.32503321  0.22012053
   0.7386085 ]
 ...
 [ 1.61808573  0.26155561  1.57647371 ...  0.74384453 -0.53663034
   0.4471109 ]
 [-0.36419542 -1.19967661 -0.30118031 ...  0.66234652 -0.39202826
   1.95832589]
 [-0.2027811  -0.09721848 -0.19853797 ...  0.45233242  1.02025202
   0.17790429]]


/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Veamos la salida del modelo para diferentes entradas:

In [4]:
print("Salidas de los modelos para las primeras 5 entradas:")
for index, test_value in enumerate(test_values):
    hash_key = hashed_values[index]
    tree_prediction = model_outputs[hash_key].get('tree', 'No disponible')
    svc_prediction = model_outputs[hash_key].get('svc', 'No disponible')
    
    print(f"\nPara la entrada: {test_value}")
    print(f"El modelo tree predice: {tree_prediction}")
    print(f"El modelo svc predice: {svc_prediction}")

print("\nSe han mostrado las predicciones para las primeras 5 entradas.")

Salidas de los modelos para las primeras 5 entradas:

Para la entrada: ['9.606', '16.84', '61.64', '280.5', '0.08481', '0.09228', '0.08422', '0.02292', '0.2036', '0.07125', '0.1844', '0.9429', '1.429', '12.07', '0.005954', '0.03471', '0.05028', '0.00851', '0.0175', '0.004031', '10.75', '23.07', '71.25', '353.6', '0.1233', '0.3416', '0.4341', '0.0812', '0.2982', '0.09825']
El modelo tree predice: Benigno
El modelo svc predice: Benigno

Para la entrada: ['20.6', '29.33', '140.1', '1265.0', '0.1178', '0.277', '0.3514', '0.152', '0.2397', '0.07016', '0.726', '1.595', '5.772', '86.22', '0.006522', '0.06158', '0.07117', '0.01664', '0.02324', '0.006185', '25.74', '39.42', '184.6', '1821.0', '0.165', '0.8681', '0.9387', '0.265', '0.4087', '0.124']
El modelo tree predice: Maligno
El modelo svc predice: Maligno

Para la entrada: ['10.48', '14.98', '67.49', '333.6', '0.09816', '0.1013', '0.06335', '0.02218', '0.1925', '0.06915', '0.3276', '1.127', '2.564', '20.77', '0.007364', '0.03867', '0.05263

# Representemos graficamente a estos datos
test_values_df = pd.DataFrame(test_values)
test_values_df = pd.concat([test_values_df, pd.Series(model_output)], axis=1)
test_values_df.columns=["0", "1", "2", "3", "label"]
sns.scatterplot(x="2", y="3", hue="label", data=test_values_df)
plt.title("Model output for test values");